# Setup and Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set
import warnings
import re
from pandas.io import gbq
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from scipy import sparse

# Get the Data

In [2]:
chicago_data = """
                    SELECT unique_key, date, primary_type, location_description, 
                            arrest, domestic, community_area,year, latitude, longitude,
                    FROM `gdac-327115.Chicago.chicago2`
                    WHERE year >= 2011
               """
chicago_data = gbq.read_gbq(chicago_data, project_id="gdac-327115")

In [3]:
#Read in an Excel file with a one to one mapping between Chicago community areas and districts
chicago_districts = pd.read_excel("ChicagoCommunityAreas.xlsx")

In [4]:
#Data type can't be joined on an int
chicago_districts.community_area = chicago_districts["community_area"].astype("string")
chicago_data.community_area = chicago_data["community_area"].astype("string")

In [5]:
#Outer join the two data sets
chicago = chicago_data.merge(chicago_districts, how="outer", left_on="community_area", right_on="community_area", )

In [6]:
#Check data types
chicago.dtypes

unique_key                            Int64
date                    datetime64[ns, UTC]
primary_type                         object
location_description                 object
arrest                              boolean
domestic                            boolean
community_area                       string
year                                  Int64
latitude                            float64
longitude                           float64
district_name                        object
community_name                       object
dtype: object

In [7]:
#Change the object data types to categorical
chicago.loc[:, chicago.dtypes == "object"] = chicago.select_dtypes(["object"]).apply(lambda x: x.astype("category"))

In [8]:
#Drop the community area variable since we have a community name variable
chicago.drop("community_area", axis = 1, inplace = True)

### Explore first few rows

In [10]:
chicago.head()

,unique_key,date,primary_type,location_description,arrest,domestic,year,latitude,longitude,district_name,community_name
0,11228094,2017-11-20 15:00:00+00:00,DECEPTIVE PRACTICE,POOL ROOM,False,False,2017,NaN,NaN,Far North,NORTH PARK
1,12135292,2020-08-13 12:00:00+00:00,DECEPTIVE PRACTICE,BANK,False,False,2020,41.993402,-87.712280,Far North,NORTH PARK
2,8905294,2012-11-27 17:30:00+00:00,PUBLIC PEACE VIOLATION,RESTAURANT,False,False,2012,41.978265,-87.708742,Far North,NORTH PARK
3,8805120,2012-09-17 00:30:00+00:00,DECEPTIVE PRACTICE,TAXICAB,False,False,2012,41.985277,-87.713827,Far North,NORTH PARK
4,9030552,2013-02-21 00:00:00+00:00,CRIM SEXUAL ASSAULT,RESIDENCE,False,True,2013,41.994323,-87.720213,Far North,NORTH PARK


In [11]:
#Check the number of rows and columns
chicago.shape

(2971810, 11)

# Split the Data into a Training Set and Test Set

In [35]:
chicago_train = chicago.loc[chicago["year"] != 2021]
chicago_test = chicago.loc[chicago["year"] == 2021]

In [20]:
#Check if the test set contains data from the full year
chicago_test.loc[:, "date"].dt.month. \
                                value_counts(). \
                                reset_index(). \
                                rename(columns={"index":"Month", "date":"Count"}). \
                                sort_values(by = "Month")

,Month,Count
5,1,16008
8,2,12852
6,3,15709
7,4,15279
4,5,17493
2,6,18468
0,7,18898
3,8,18110
1,9,18632
9,10,5507


# Data Exploration and Cleaning

In [22]:
#Check the total number of missing values for each variable
chicago_train.isnull().sum()

unique_key                  0
date                        0
primary_type                0
location_description     7844
arrest                      0
domestic                    0
year                        0
latitude                26843
longitude               26843
district_name             218
community_name            218
dtype: int64

In [21]:
#Check to make sure there are no duplicate incidents based on the "unique key" variable
assert len(chicago_train["unique_key"].unique()) == chicago_train.shape[0]

### Cleaning Primary Type Variable

In [23]:
#Quick reference on the number of missing values
chicago_train["primary_type"].isnull().sum()

0

In [10]:
#Examine the different crime types listed
chicago_train["primary_type"].value_counts()

THEFT                                636007
BATTERY                              512318
CRIMINAL DAMAGE                      299943
NARCOTICS                            222406
ASSAULT                              189549
OTHER OFFENSE                        171368
DECEPTIVE PRACTICE                   167709
BURGLARY                             152532
MOTOR VEHICLE THEFT                  120013
ROBBERY                              108092
CRIMINAL TRESPASS                     69971
WEAPONS VIOLATION                     45871
OFFENSE INVOLVING CHILDREN            23359
PUBLIC PEACE VIOLATION                21829
PROSTITUTION                          12438
CRIM SEXUAL ASSAULT                   12334
INTERFERENCE WITH PUBLIC OFFICER      11793
SEX OFFENSE                           10771
HOMICIDE                               5669
ARSON                                  4482
GAMBLING                               3507
LIQUOR LAW VIOLATION                   3407
CRIMINAL SEXUAL ASSAULT         

It appears "CRIM SEXUAL ASSAULT" should be merged into "CRIMINAL SEXUAL ASSAULT". We'll also merge the three various "Non-criminal" categories into one category. We'll do the same thing for the "OTHER NARCOTIC VIOLATION" category. Finally, we'll move "RITUALISM" into the "OTHER" since it has only one occurence. Let's collect all of these changes into one function. 

In [11]:
def crime_type_cleaner(df):
    """
    This function cleans the primary_type column in place. 
    
    df: DataFrame with primary_type column
    
    returns: None
    """
    df.loc[:, "primary_type"].replace(["CRIM SEXUAL ASSAULT"], "CRIMINAL SEXUAL ASSAULT", inplace = True)

    df.loc[:, "primary_type"].replace(["OTHER NARCOTIC VIOLATION"], "NARCOTICS", inplace = True)

    df.loc[:, "primary_type"].replace(["NON - CRIMINAL", "NON-CRIMINAL (SUBJECT SPECIFIED)"], "NON-CRIMINAL", inplace = True)

    df.loc[:, "primary_type"].replace(["RITUALISM"], "OTHER OFFENSE", inplace = True)
   

In [29]:
#Clean the primary type variable
crime_type_cleaner(chicago_train)
#Check the cleaned variable
chicago_train["primary_type"].value_counts()

THEFT                                636007
BATTERY                              512318
CRIMINAL DAMAGE                      299943
NARCOTICS                            222467
ASSAULT                              189549
OTHER OFFENSE                        171369
DECEPTIVE PRACTICE                   167709
BURGLARY                             152532
MOTOR VEHICLE THEFT                  120013
ROBBERY                              108092
CRIMINAL TRESPASS                     69971
WEAPONS VIOLATION                     45871
OFFENSE INVOLVING CHILDREN            23359
PUBLIC PEACE VIOLATION                21829
CRIMINAL SEXUAL ASSAULT               14850
PROSTITUTION                          12438
INTERFERENCE WITH PUBLIC OFFICER      11793
SEX OFFENSE                           10771
HOMICIDE                               5669
ARSON                                  4482
GAMBLING                               3507
LIQUOR LAW VIOLATION                   3407
KIDNAPPING                      

### Cleaning Location Description Variable

In [13]:
#Check the number of missing values
chicago_train["location_description"].isnull().sum()

7844

In [14]:
#Look at which rows are missing
chicago_train.loc[chicago_train["location_description"].isnull()]

,unique_key,date,primary_type,location_description,arrest,domestic,year,latitude,longitude,district_name,community_name
192,10625982,2015-08-01 16:20:00+00:00,DECEPTIVE PRACTICE,NaN,False,False,2015,NaN,NaN,Far North,NORTH PARK
285,12185129,2020-09-28 21:37:00+00:00,DECEPTIVE PRACTICE,NaN,False,False,2020,41.987639,-87.710214,Far North,NORTH PARK
2019,11489624,2018-10-26 15:05:00+00:00,DECEPTIVE PRACTICE,NaN,False,False,2018,41.994360,-87.723939,Far North,NORTH PARK
2034,12018775,2020-03-26 14:00:00+00:00,DECEPTIVE PRACTICE,NaN,False,False,2020,41.974920,-87.706215,Far North,NORTH PARK
2042,10596491,2016-06-16 13:00:00+00:00,DECEPTIVE PRACTICE,NaN,False,False,2016,41.985369,-87.712605,Far North,NORTH PARK
...,...,...,...,...,...,...,...,...,...,...,...
2970607,10465221,2016-03-24 14:25:00+00:00,DECEPTIVE PRACTICE,NaN,False,False,2016,41.982365,-87.662518,Far North,EDGEWATER
2970615,12114559,2020-07-23 11:58:00+00:00,DECEPTIVE PRACTICE,NaN,False,False,2020,41.991336,-87.656851,Far North,EDGEWATER
2970618,10883083,2017-03-19 11:55:00+00:00,DECEPTIVE PRACTICE,NaN,False,False,2017,NaN,NaN,Far North,EDGEWATER
2970632,10852091,2017-02-16 17:15:00+00:00,DECEPTIVE PRACTICE,NaN,False,False,2017,41.982755,-87.662528,Far North,EDGEWATER


Looks like there might be some relationship between the missing locations and the type of crime committed. Let's further explore this. 

In [36]:
#Look at the counts of the primary type variable with the data filtered to those values of location that are missing
chicago_train.loc[chicago_train["location_description"].isnull()]["primary_type"].value_counts()

DECEPTIVE PRACTICE                   7124
THEFT                                 704
BURGLARY                                5
ROBBERY                                 4
BATTERY                                 2
CRIMINAL DAMAGE                         2
OTHER OFFENSE                           1
ARSON                                   1
CRIMINAL SEXUAL ASSAULT                 1
SEX OFFENSE                             0
NON-CRIMINAL                            0
RITUALISM                               0
PUBLIC PEACE VIOLATION                  0
PUBLIC INDECENCY                        0
PROSTITUTION                            0
STALKING                                0
OTHER NARCOTIC VIOLATION                0
OFFENSE INVOLVING CHILDREN              0
OBSCENITY                               0
NON-CRIMINAL (SUBJECT SPECIFIED)        0
MOTOR VEHICLE THEFT                     0
NON - CRIMINAL                          0
NARCOTICS                               0
ASSAULT                           

It appears that nearly all missing values for location description have a primary type that is labeled as "DECEPTIVE PRACTICE". Let's see what the non-missing values for location description have for their corresponding "DECEPTIVE PRACTICE" values.

In [24]:
#Create a boolean mask that filters the data to non-missing locations with a primary type of "DECEPTIVE PRACTICE"
mask = (chicago_train["location_description"].notnull()) & (chicago_train["primary_type"] == "DECEPTIVE PRACTICE")
chicago_train.loc[mask]["location_description"].value_counts()

RESIDENCE                54361
APARTMENT                17232
OTHER                    16680
STREET                    8369
BANK                      6910
                         ...  
PORCH                        0
POOLROOM                     0
EXPRESSWAY EMBANKMENT        0
FARM                         0
YMCA                         0
Name: location_description, Length: 197, dtype: int64

Thus, it appears that the "RESIDENCE" location occurrs the most for non-missing "DECEPTIVE PRACTICE" crimes. Therefore, we can impute the missing values for location that have the corresponding crime as "DECEPTIVE PRACTICE" with "RESIDENCE".  

Let's do the same process for the "THEFT" crime

In [23]:
#Create a boolean mask that filters the data to non-missing locations with a primary type of "THEFT"
mask = (chicago_train["location_description"].notnull()) & (chicago_train["primary_type"] == "THEFT")
chicago_train.loc[mask]["location_description"].value_counts()

STREET                152284
RESIDENCE              58720
APARTMENT              41793
SMALL RETAIL STORE     35004
DEPARTMENT STORE       32695
                       ...  
ELEVATOR                   0
DRIVEWAY                   0
SCHOOL YARD                0
CTA PROPERTY               0
YMCA                       0
Name: location_description, Length: 197, dtype: int64

Now, it appears that the "STREET" location is the most occurring for non-missing "THEFT" crimes. Let's impute the missing values for location that have the corresponding crime as "THEFT" with "STREET". 

Finally, we'll collect the remaining crimes into one group due to their low counts and check what crime is most occurring for the group.

In [37]:
crime_type = ["BURGLARY", "ROBBERY", "BATTERY", "CRIMINAL DAMAGE", "ARSON", "CRIMINAL SEXUAL ASSAULT", "OTHER OFFENSE"]
mask = (chicago_train["primary_type"].isin(crime_type)) & (chicago_train["location_description"].notnull())
chicago_train.loc[mask, ["location_description"]].value_counts()

location_description
RESIDENCE               277616
APARTMENT               248008
STREET                  235210
SIDEWALK                125040
RESIDENCE-GARAGE         35372
                         ...  
PARKING LOT                  0
CLUB                         0
OFFICE                       0
NURSING HOME                 0
YMCA                         0
Length: 197, dtype: int64

Again, it looks like "RESIDENCE" is the most occuring location for the group of crimes when location is not missing. Thus, we can impute the missing location values for the group of crimes as "RESIDENCE". 

We'll collect all of these missing value imputations into one function

In [38]:
def location_imputer(df):
    """
    This function imputes missing values in the location_description column by setting the missing value to the value that 
    occurs most in the primary_type column in place
    
    df: DataFrame that contains the location_description column
    
    returns: None
    """
    mask = (df.loc[:, "primary_type"] == "DECEPTIVE PRACTICE") & (df.loc[:, "location_description"].isnull())
    df.loc[mask, ["location_description"]] = "RESIDENCE"
    
    mask = (df.loc[:, "primary_type"] == "THEFT") & (df.loc[:, "location_description"].isnull())
    df.loc[mask, ["location_description"]] = "STREET"
    
    crime_type = ["BURGLARY", "ROBBERY", "BATTERY", "CRIMINAL DAMAGE", "ARSON", "CRIMINAL SEXUAL ASSAULT", "OTHER OFFENSE"]
    mask = (df.loc[:, "primary_type"].isin(crime_type)) & (df.loc[:, "location_description"].isnull())
    df.loc[mask, ["location_description"]] = "RESIDENCE"
    

In [40]:
#Impute the missing values
location_imputer(chicago_train)
#Check that there are no missing values
chicago_train["location_description"].isnull().sum()

0